# 0- Import the packages and reading data

In [1]:
# data manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)
import datetime
# ploting
from matplotlib import style
style.use('ggplot')
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# modeling
'''
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.tsa.arima_process as sta
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.statespace as sts
'''
import warnings
warnings.filterwarnings(action='once')
import sys

# data reading
sales_train_validation = pd.read_csv('data/sales_train_validation.csv')
calendar = pd.read_csv('data/calendar.csv',parse_dates=[0])
sell_prices = pd.read_csv('data/sell_prices.csv')



**WE FIRST WILL ONLY TAKE 20 % OF THE DATA TO EASE THE LOAD ON CPU/RAM **


In [2]:
sales_train_validation = sales_train_validation.sample(frac=0.2)

In [3]:
#reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

sales_train_validation = reduce_mem_usage(sales_train_validation)
calendar = reduce_mem_usage(calendar)
sell_prices = reduce_mem_usage(sell_prices)

Memory usage after optimization is: 12.62 MB
Decreased by 85.9%
Memory usage after optimization is: 0.12 MB
Decreased by 41.9%
Memory usage after optimization is: 130.48 MB
Decreased by 37.5%


# 1- Data Exploration
## 1-a) Nan Values
* Nan values are essentially in the calendar datasets in the 'event_name_1', 'event_type_1', 'event_name_2','event_type_2' columns.
  We suppose that nan here corresponds to day without any special event 
* The dataframe is however **very sparse** (lots of zeros as stated in the guidelines = > intermittency and sporadic demand )
  On average, **68 %are zeros** and 50 % have more than 73% of zeros in their series ( median )

In [4]:
# missing values examination
print("sales_train_validation nan values: ",sales_train_validation.isna().sum().sum())
print("sell_prices nan values: ", sell_prices.isna().sum().sum())
print("calendar nan values: ", calendar.isna().sum().sum())
print() #line break 

print(calendar.isna().sum())
print() #line break 

# data sparcity check
percentage_zero=(sales_train_validation==0).sum(axis=1)/1913 ## 1913 is the number of days 
print(percentage_zero.describe())

sales_train_validation nan values:  0
sell_prices nan values:  0
calendar nan values:  7542

date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

count    6098.000000
mean        0.678290
std         0.226682
min         0.002614
25%         0.531626
50%         0.733926
75%         0.864088
max         0.992682
dtype: float64


## 1-b) Time series 

In [5]:
value_vars=sales_train_validation.columns.to_list()[6:]  # remove the first 6 variables ie 'id','item_id', 'dept_id', 'cat_id', 'store_id','state_id'
id_vars=['id','item_id', 'dept_id', 'cat_id', 'store_id','state_id']

# unpivoting the columns 
sales = pd.melt(sales_train_validation,id_vars=id_vars, value_vars=value_vars,var_name='d', value_name='sales_count')
# joining with calendar but memomry run out with this method 
sales=pd.merge(sales,calendar,left_on='d',right_on='d',how="left")

In [6]:
# plot by month and year
sales['month_year'] = sales['date'].apply(lambda x: x.replace(day=1)) 
# source : https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-separately-from-pandas-datetime-column


In [7]:
#cleaning
del id_vars,value_vars,sales_train_validation

## i. random plot
* 2 same item_id can have different starting date and different bahaviours across different state
* data before starting date NOT to take into account => ROLLING FEATURES


In [ ]:
n = 2 # number of item_id to be plotted  
plotted_sales = sales[sales['item_id'].isin(sales.item_id.tolist()[:n])]

fig = px.line(plotted_sales, x="date", y="sales_count",
        line_shape="spline", render_mode="svg",color="id")
fig.show()

## ii.aggregating sales by 
* date
  * upward seasonal trend 
  * 0 sales for christmas with some exceptions most likely data input errors => to clean ? 
* state/store/product
  * CA sales are higher (4 stores vs 3 in other states) 
  * across stores seasonal effects are different : ex in WI2 only, we see a significant        drop in Food sales at the end of each month (10 days) => feature  ? 
  * lowest on monday higest on thursday/friday
  *   

In [ ]:
# total sales
sales.groupby(by='date').sum()[['sales_count']].plot()

# christmas sales 
sales.groupby(by='date').sum()['sales_count'].sort_values()
# sales[(sales.event_name_1=='Christmas') & (sales.sales_count!=0)] # list is here

# store sales
fig = px.line(sales.groupby(by=['date','store_id'], as_index=False).sum(), x="date", y="sales_count",# color="continent", line_group="country", hover_name="country",
        line_shape="spline", render_mode="svg",color="store_id")
fig.show()
#stores and categories 
fig = px.line(sales.groupby(by=['date','cat_id','store_id'], as_index=False).sum(), x="date", y="sales_count", facet_row="store_id", color="cat_id"
                 #,category_orders={"cat_id": ["Thur", "Fri", "Sat", "Sun"], "store_id": ["TX_2","TX_1","CA_3","WI_2","TX_3","CA_4","WI_1","WI_3","CA_2","CA_1"]}
                ,width=800, height=1400)
fig.show()

fig = px.line(sales.groupby(by=['month_year','dept_id','state_id'], as_index=False).sum(), x="month_year", y="sales_count", facet_row="state_id",facet_col="dept_id"
                 ,width=1200, height=700)
fig.show()

In [ ]:
# seperate test dataframes
sales_validation_id = [row for row in submission['id'] if 'validation' in row]
sales_evaluation_id = [row for row in submission['id'] if 'evaluation' in row]

sales_validation = submission[submission['id'].isin(sales_validation_id)]
sales_evaluation = submission[submission['id'].isin(sales_evaluation_id)]